In [13]:
import requests
import json

from apikeys import apikey_coinio as key

import sqlite3
from sqlite3 import Error

import glob
import pandas as pd

from sqlalchemy import create_engine

In [14]:
def initialise_db():
    database = r"crypto_db.sqlite"
    
    sql_create_assets_table = """ CREATE TABLE IF NOT EXISTS "assets" (
        "asset_id" VARCHAR   NOT NULL,
        "name" VARCHAR   NOT NULL,
        "type_is_crypto" INT   NOT NULL,
        "data_quote_start" VARCHAR   NOT NULL,
        "data_quote_end" VARCHAR   NOT NULL,
        "data_orderbook_start" VARCHAR   NOT NULL,
        "data_orderbook_end" VARCHAR   NOT NULL,
        "data_trade_start" VARCHAR   NOT NULL,
        "data_trade_end" VARCHAR   NOT NULL,
        "data_quote_count" VARCHAR   NOT NULL,
        "data_trade_count" VARCHAR   NOT NULL,
        "data_symbols_count" INT   NOT NULL,
        "volume_1hrs_usd" FLOAT   NOT NULL,
        "volume_1day_usd" FLOAT   NOT NULL,
        "volume_1mth_usd" FLOAT   NOT NULL,
        "price_usd" FLOAT   NOT NULL,
        PRIMARY KEY ("asset_id"),
        FOREIGN KEY ("asset_id") REFERENCES "historic_trades" ("asset_id")
    );"""

    sql_create_periods_table = """ CREATE TABLE IF NOT EXISTS "periods" (
        "period_id" VARCHAR   NOT NULL,
        "length_seconds" INT   NOT NULL,
        "length_months" INT   NOT NULL,
        "unit_count" INT   NOT NULL,
        "unit_name" VARCHAR   NOT NULL,
        "display_name" VARCHAR   NOT NULL
    );"""

    sql_create_current_rates_table = """ CREATE TABLE IF NOT EXISTS "current_rates" (
        "time" VARCHAR   NOT NULL,
        "asset_id_base" VARCHAR   NOT NULL,
        "asset_id_quote" VARCHAR   NOT NULL,
        "rate" FLOAT   NOT NULL,
        FOREIGN KEY ("asset_id_base") REFERENCES "assets" ("asset_id")
    );"""

    sql_create_exchanges_table = """ CREATE TABLE IF NOT EXISTS "exchanges" (
        "exchange_id" VARCHAR   NOT NULL,
        "website" VARCHAR   NOT NULL,
        "name" VARCHAR   NOT NULL,
        "data_start" VARCHAR   NOT NULL,
        "data_end" VARCHAR   NOT NULL,
        "data_quote_start" VARCHAR   NOT NULL,
        "data_quote_end" VARCHAR   NOT NULL,
        "data_orderbook_start" VARCHAR   NOT NULL,
        "data_orderbook_end" VARCHAR   NOT NULL,
        "data_trade_start" VARCHAR   NOT NULL,
        "data_trade_end" VARCHAR   NOT NULL,
        "data_symbols_count" INT   NOT NULL,
        "volume_1hrs_usd" FLOAT   NOT NULL,
        "volume_1day_usd" FLOAT   NOT NULL,
        "volume_1mth_usd" FLOAT   NOT NULL
    );"""

    sql_create_historic_trades_table = """ CREATE TABLE IF NOT EXISTS "historic_trades" (
        "asset_id" VARCHAR  NOT NULL,
        "time_period_start" VARCHAR   NOT NULL,
        "time_period_end" VARCHAR   NOT NULL,
        "time_open" VARCHAR   NULL,
        "time_close" VARCHAR   NULL,
        "price_open" FLOAT   NOT NULL,
        "price_high" FLOAT   NOT NULL,
        "price_low" FLOAT   NOT NULL,
        "price_close" FLOAT   NOT NULL,
        "volume_traded" FLOAT   NOT NULL,
        "trades_count" INT   NULL
    );"""
    
    def create_connection(db_file):
        """ create a database connection to the SQLite database specified by db_file
        :param db_file: database file
        :return: Connection object or None
        """
        conn = None
        try:
            conn = sqlite3.connect(db_file)
            return conn
        except Error as e:
            print(e)
        return conn
    # END create_connection

    def execute_sql_cmd(conn, command):
        """ run a sql command statement
        :param conn: Connection object
        :param execute_sql_cmd: run sql statement
        :return:
        """
        try:
            c = conn.cursor()
            c.execute(command)
        except Error as e:
            print(e)
    # END execute_sql_cmd

    conn = create_connection(database)
    
    if conn is not None:
        execute_sql_cmd(conn, sql_create_assets_table)
        execute_sql_cmd(conn, sql_create_periods_table)
        execute_sql_cmd(conn, sql_create_current_rates_table)
        execute_sql_cmd(conn, sql_create_exchanges_table)
        execute_sql_cmd(conn, sql_create_historic_trades_table)

    else:
        print("Error! cannot create the database connection.")
    conn.close()
# END initialise_db


def data_get():
    find_csvs = glob.glob("data_raw/*.csv")

    for i in find_csvs:
        df_csv = pd.read_csv(i)
        df_csv['asset_id'] = find_csvs[find_csvs.index(i)].split("\\", 1)[1].split("_", 2)[0]
        df_csv.dropna(inplace = True)
        df_csv = df_csv.drop(columns=['Adj Close'])
        df_csv = df_csv.rename(columns ={
            'Date' : 'time_period_end',
            'Open' : 'price_open',
            'High' : 'price_high',
            'Low' : 'price_low',
            'Close' : 'price_close',
            'Volume' : 'volume_traded',
        })
        df_csv["time_period_end"] = pd.to_datetime(df_csv["time_period_end"],format='%d/%m/%Y')

#         with open(f'data_raw/{df_csv.asset_id[0]}.json', 'w') as f:
#             f.write(df_csv.to_json())
#         with open( f'data_raw/{df_csv.asset_id[0]}.json', 'r' ) as jj:
#             json_d = json.load( jj )
#             df = pd.DataFrame( json_d )
#         display(df)
# END data_get














def data_clean():
    pass
# END data_clean

def data_update():
    pass
# END data_update

initialise_db()
#data_get()

In [15]:
find_csvs = glob.glob("data_raw/*.csv")

for i in find_csvs:
    df_csv = pd.read_csv(i)
    df_csv['asset_id'] = find_csvs[find_csvs.index(i)].split("\\", 1)[1].split("_", 2)[0]
    #df_csv['Date'] = df_csv['Date'].str.replace('/', '-')
    #YMD
    df_csv.dropna(inplace = True)
    df_csv = df_csv.drop(columns=['Adj Close'])
    df_csv = df_csv.rename(columns ={
        'Date' : 'time_period_end',
        'Open' : 'price_open',
        'High' : 'price_high',
        'Low' : 'price_low',
        'Close' : 'price_close',
        'Volume' : 'volume_traded',
    })
    
    df_csv['time_period_end'] = pd.to_datetime(df_csv['time_period_end'],format='%d/%m/%Y')
#     df_csv['time_period_start'] = df_csv['time_period_end'].apply(pd.DateOffset(-1))
    df_csv['time_period_start'] = df_csv['time_period_end'].copy()
    
    #display(df_csv)
    
    with open(f'data_raw/{df_csv.asset_id[0]}.json', 'w') as f:
        f.write(df_csv.to_json())
    
    con = sqlite3.connect("crypto_db.sqlite")

#     df_csv.to_sql("historic_trades", con, if_exists="append", index=False)
    df = df_csv.head(50)
    df.to_sql("historic_trades", con, if_exists="append", index=False)
    #display(df)

    con.close()


In [16]:
with open( 'data_raw/BTC.json', 'r' ) as jj: #open api results    
    json_d = json.load( jj )
    df_api = pd.DataFrame( json_d ) #save to dataframe
df_api['asset_id'] = 'BTC'
df = df_api.head(50)

con = sqlite3.connect("crypto_db.sqlite")
df.to_sql("historic_trades", con, if_exists="append", index=False)
con.close()

In [17]:
with open( 'data_raw/ETH.json', 'r' ) as jj: #open api results    
    json_d = json.load( jj )
    df_api = pd.DataFrame( json_d ) #save to dataframe
df_api['asset_id'] = 'ETH'
df = df_api.head(50)

con = sqlite3.connect("crypto_db.sqlite")
df.to_sql("historic_trades", con, if_exists="append", index=False)
con.close()

In [18]:
con = sqlite3.connect("crypto_db.sqlite")

a = pd.read_sql_query("SELECT * from historic_trades WHERE asset_id='brentoil'", con)
b = pd.read_sql_query(f"SELECT * from historic_trades WHERE asset_id='goldfutures'", con)
c = pd.read_sql_query("SELECT * from historic_trades WHERE asset_id='BTC'", con)
d = pd.read_sql_query(f"SELECT * from historic_trades WHERE asset_id='ETH'", con)
# df_csv.to_sql("historic_trades", con, if_exists="replace")

con.close()

display(a, b, c , d)

,asset_id,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count
0,brentoil,2010-01-04 00:00:00,2010-01-04 00:00:00,None,None,80.120003,79.820000,80.120003,80.120003,97.0,None
1,brentoil,2010-01-05 00:00:00,2010-01-05 00:00:00,None,None,80.589996,80.260002,80.589996,80.589996,97.0,None
2,brentoil,2010-01-06 00:00:00,2010-01-06 00:00:00,None,None,81.129997,81.699997,80.129997,81.889999,7.0,None
3,brentoil,2010-01-07 00:00:00,2010-01-07 00:00:00,None,None,81.510002,81.510002,81.629997,81.510002,7.0,None
4,brentoil,2010-01-08 00:00:00,2010-01-08 00:00:00,None,None,81.370003,81.529999,81.089996,81.370003,7.0,None
5,brentoil,2010-01-11 00:00:00,2010-01-11 00:00:00,None,None,80.970001,81.709999,81.010002,80.970001,7.0,None
6,brentoil,2010-01-12 00:00:00,2010-01-12 00:00:00,None,None,79.300003,79.300003,79.029999,79.300003,7.0,None
7,brentoil,2010-01-13 00:00:00,2010-01-13 00:00:00,None,None,78.529999,79.019997,77.250000,78.309998,18.0,None
8,brentoil,2010-01-14 00:00:00,2010-01-14 00:00:00,None,None,77.820000,78.470001,78.010002,77.820000,6.0,None
9,brentoil,2010-01-15 00:00:00,2010-01-15 00:00:00,None,None,77.550003,77.820000,76.879997,77.110001,4.0,None


,asset_id,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count
0,goldfutures,2010-01-04 00:00:00,2010-01-04 00:00:00,None,None,1117.699951,1122.300049,1097.099976,1117.699951,184.0,None
1,goldfutures,2010-01-05 00:00:00,2010-01-05 00:00:00,None,None,1118.099976,1126.500000,1115.000000,1118.099976,53.0,None
2,goldfutures,2010-01-06 00:00:00,2010-01-06 00:00:00,None,None,1135.900024,1139.199951,1120.699951,1135.900024,363.0,None
3,goldfutures,2010-01-07 00:00:00,2010-01-07 00:00:00,None,None,1133.099976,1133.099976,1129.199951,1133.099976,56.0,None
4,goldfutures,2010-01-08 00:00:00,2010-01-08 00:00:00,None,None,1138.199951,1138.199951,1122.699951,1138.199951,54.0,None
5,goldfutures,2010-01-11 00:00:00,2010-01-11 00:00:00,None,None,1150.699951,1161.199951,1143.000000,1150.699951,177.0,None
6,goldfutures,2010-01-12 00:00:00,2010-01-12 00:00:00,None,None,1128.900024,1157.199951,1127.199951,1128.900024,51.0,None
7,goldfutures,2010-01-13 00:00:00,2010-01-13 00:00:00,None,None,1136.400024,1136.400024,1121.000000,1136.400024,58.0,None
8,goldfutures,2010-01-14 00:00:00,2010-01-14 00:00:00,None,None,1137.000000,1145.900024,1132.800049,1142.599976,81.0,None
9,goldfutures,2010-01-15 00:00:00,2010-01-15 00:00:00,None,None,1132.800049,1133.400024,1127.199951,1130.099976,50.0,None


,asset_id,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count
0,BTC,2011-09-13T00:00:00.0000000Z,2011-09-14T00:00:00.0000000Z,2011-09-13T13:53:36.0000000Z,2011-09-13T19:34:36.0000000Z,5.80,6.00,5.65,5.97,58.371382,12
1,BTC,2011-09-14T00:00:00.0000000Z,2011-09-15T00:00:00.0000000Z,2011-09-14T07:00:00.0000000Z,2011-09-14T15:42:31.0000000Z,5.58,5.72,5.52,5.53,61.145984,14
2,BTC,2011-09-15T00:00:00.0000000Z,2011-09-16T00:00:00.0000000Z,2011-09-15T08:39:33.0000000Z,2011-09-15T15:11:22.0000000Z,5.12,5.24,5.00,5.13,80.140795,6
3,BTC,2011-09-16T00:00:00.0000000Z,2011-09-17T00:00:00.0000000Z,2011-09-16T07:02:06.0000000Z,2011-09-16T16:13:02.0000000Z,4.82,4.87,4.80,4.85,39.914007,4
4,BTC,2011-09-17T00:00:00.0000000Z,2011-09-18T00:00:00.0000000Z,2011-09-17T15:32:08.0000000Z,2011-09-17T15:32:08.0000000Z,4.87,4.87,4.87,4.87,0.300000,1
5,BTC,2011-09-18T00:00:00.0000000Z,2011-09-19T00:00:00.0000000Z,2011-09-18T10:06:49.0000000Z,2011-09-18T20:11:23.0000000Z,4.87,4.92,4.81,4.92,119.812800,8
6,BTC,2011-09-19T00:00:00.0000000Z,2011-09-20T00:00:00.0000000Z,2011-09-19T13:47:03.0000000Z,2011-09-19T13:47:03.0000000Z,4.90,4.90,4.90,4.90,20.000000,1
7,BTC,2011-09-20T00:00:00.0000000Z,2011-09-21T00:00:00.0000000Z,2011-09-20T06:51:41.0000000Z,2011-09-20T16:38:45.0000000Z,4.92,5.66,4.92,5.66,89.280711,17
8,BTC,2011-09-21T00:00:00.0000000Z,2011-09-22T00:00:00.0000000Z,2011-09-21T06:43:16.0000000Z,2011-09-21T16:47:28.0000000Z,5.70,5.79,5.66,5.66,17.629322,5
9,BTC,2011-09-22T00:00:00.0000000Z,2011-09-23T00:00:00.0000000Z,2011-09-22T06:19:59.0000000Z,2011-09-22T13:32:57.0000000Z,5.68,5.72,5.68,5.72,43.778422,5


,asset_id,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count
0,ETH,2016-05-09T00:00:00.0000000Z,2016-05-10T00:00:00.0000000Z,2016-05-09T13:32:02.4140000Z,2016-05-09T23:04:26.9530000Z,12.00,12.00,9.36,9.98,1317.902740,50
1,ETH,2016-06-09T00:00:00.0000000Z,2016-06-10T00:00:00.0000000Z,2016-06-09T15:33:22.5865670Z,2016-06-09T18:59:42.9223390Z,14.65,14.70,14.57,14.65,1918.261279,95
2,ETH,2016-06-10T00:00:00.0000000Z,2016-06-11T00:00:00.0000000Z,2016-06-10T18:14:42.0203860Z,2016-06-10T23:56:23.2469820Z,14.19,14.26,13.76,13.91,6847.468569,417
3,ETH,2016-06-11T00:00:00.0000000Z,2016-06-12T00:00:00.0000000Z,2016-06-11T00:03:38.6373100Z,2016-06-11T23:59:55.3640050Z,13.93,14.29,13.86,14.10,7331.345783,549
4,ETH,2016-06-12T00:00:00.0000000Z,2016-06-13T00:00:00.0000000Z,2016-06-12T00:00:58.2260200Z,2016-06-12T23:52:20.6899120Z,14.14,15.91,13.92,15.58,19734.452652,1288
5,ETH,2016-06-13T00:00:00.0000000Z,2016-06-14T00:00:00.0000000Z,2016-06-13T00:00:30.2181920Z,2016-06-13T23:52:46.7570560Z,15.91,17.93,15.49,17.89,24139.283081,1103
6,ETH,2016-06-14T00:00:00.0000000Z,2016-06-15T00:00:00.0000000Z,2016-06-14T00:00:53.3748990Z,2016-06-14T23:59:36.9962940Z,17.86,19.08,16.96,18.79,35354.001759,1845
7,ETH,2016-06-15T00:00:00.0000000Z,2016-06-16T00:00:00.0000000Z,2016-06-15T00:01:23.7828760Z,2016-06-15T23:57:29.0675410Z,18.79,18.81,17.89,18.42,13621.142096,736
8,ETH,2016-06-16T00:00:00.0000000Z,2016-06-17T00:00:00.0000000Z,2016-06-16T00:09:22.4615360Z,2016-06-16T23:58:50.6226720Z,18.51,21.35,18.50,20.99,34254.401549,1753
9,ETH,2016-06-17T00:00:00.0000000Z,2016-06-18T00:00:00.0000000Z,2016-06-17T00:02:48.5103960Z,2016-06-17T23:58:47.0825940Z,20.99,22.00,13.75,15.65,120860.822954,3928
